In [4]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langgraph.checkpoint.memory import MemorySaver
import time

In [5]:
class CrashState(TypedDict):
  input: str
  step1: str
  step2: str
  step3: str

In [6]:
# 2. Define steps
def step_1(state: CrashState) -> CrashState:
  print("Step 1 executed")
  return {"step1": "done", "input": state["input"]}

def step_2(state: CrashState) -> CrashState:
  print(" Step 2 hanging ... now manually interrupt from the notebook toolbar (STOP button)")
  time.sleep(30) # Simulate long-running hang
  return {"step2": "done"}

def step_3(state: CrashState) -> CrashState:
  print("Step 3 executed")
  return {"step3": "done"}


In [7]:
builder = StateGraph(CrashState)

builder.add_node('step_1', step_1)
builder.add_node('step_2', step_2)
builder.add_node('step_3', step_3)

builder.add_edge(START, 'step_1')
builder.add_edge('step_1', 'step_2')
builder.add_edge('step_2', 'step_3')
builder.add_edge('step_3', END)

checkpointer = MemorySaver()
graph = builder.compile(checkpointer=checkpointer)



In [8]:
config1 = {'configurable' : {'thread_id': '1'}}
try:
  print("Running, please interupt manually during step_2")
  graph.invoke({'input':'start'}, config= config1)
except KeyboardInterrupt:
  print('Karnel manually interrupted! Crash simulated.')

Running, please interupt manually during step_2
Step 1 executed
 Step 2 hanging ... now manually interrupt from the notebook toolbar (STOP button)
Karnel manually interrupted! Crash simulated.


In [9]:
graph.get_state(config1)

StateSnapshot(values={'input': 'start', 'step1': 'done'}, next=('step_2',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f1096f4-18e3-6d8c-8001-9906b25de928'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}}, created_at='2026-02-14T06:34:47.105678+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f1096f4-18df-6744-8000-b6dfb1c4677b'}}, tasks=(PregelTask(id='33aa2868-0f62-99f8-fe45-6b7572582063', name='step_2', path=('__pregel_pull', 'step_2'), error=None, interrupts=(), state=None, result=None),), interrupts=())

In [10]:
list(graph.get_state_history(config1))

[StateSnapshot(values={'input': 'start', 'step1': 'done'}, next=('step_2',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f1096f4-18e3-6d8c-8001-9906b25de928'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}}, created_at='2026-02-14T06:34:47.105678+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f1096f4-18df-6744-8000-b6dfb1c4677b'}}, tasks=(PregelTask(id='33aa2868-0f62-99f8-fe45-6b7572582063', name='step_2', path=('__pregel_pull', 'step_2'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'input': 'start'}, next=('step_1',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f1096f4-18df-6744-8000-b6dfb1c4677b'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2026-02-14T06:34:47.103878+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f1096f4-18dd

In [12]:
# resume, or re-run the workflow - 
graph.invoke(None, config=config1)

 Step 2 hanging ... now manually interrupt from the notebook toolbar (STOP button)
Step 3 executed


{'input': 'start', 'step1': 'done', 'step2': 'done', 'step3': 'done'}

In [14]:
list(graph.get_state_history(config1))

[StateSnapshot(values={'input': 'start', 'step1': 'done', 'step2': 'done', 'step3': 'done'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f1096fd-2edb-60eb-8003-64f9f02a514c'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}}, created_at='2026-02-14T06:38:51.000855+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f1096fd-2ed7-64e9-8002-6ad78bb75d69'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'input': 'start', 'step1': 'done', 'step2': 'done'}, next=('step_3',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f1096fd-2ed7-64e9-8002-6ad78bb75d69'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2026-02-14T06:38:50.999319+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f1096f4-18e3-6d8c-8001-9906b25de928'}}, tasks=(PregelTask(id='ab135129-e5dd-8a6f-cf81-9fd4e0be58ef', name='s